In [ ]:
# !pip install pypianoroll music21

In [1]:
import os
import numpy as np
import torch
from pathlib import Path
import music21
from IPython.display import Audio, display
import librosa

def load_npz_file(file_path):
    """加载.npz文件并提取钢琴轨道和和弦轨道"""
    data = np.load(file_path)
    # 假设数据格式为 [track, time, pitch]
    # 轨道1是钢琴轨道，轨道2是和弦轨道
    piano_track = data['arr_0'][1]  # 钢琴轨道
    chord_track = data['arr_0'][2]  # 和弦轨道
    return piano_track, chord_track

def print_track_info(piano_track, chord_track):
    """打印轨道信息"""
    print("钢琴轨道形状:", piano_track.shape)
    print("和弦轨道形状:", chord_track.shape)
    
    # 打印每个时间步的音符数量
    print("\n钢琴轨道音符统计:")
    for t in range(min(10, len(piano_track))):  # 只显示前10个时间步
        notes = np.where(piano_track[t] > 0)[0]
        print(f"时间步 {t}: {len(notes)} 个音符")
    
    print("\n和弦轨道音符统计:")
    for t in range(min(10, len(chord_track))):  # 只显示前10个时间步
        notes = np.where(chord_track[t] > 0)[0]
        print(f"时间步 {t}: {len(notes)} 个音符")

def analyze_sequence(piano_track, chord_track, start_time=0, length=32):
    """分析指定时间段的音符"""
    piano_sequence = piano_track[start_time:start_time + length]
    chord_sequence = chord_track[start_time:start_time + length]
    
    print("\n时间步分析:")
    for t in range(length):
        # 获取钢琴音符
        piano_notes = np.where(piano_sequence[t] > 0)[0]
        # 获取和弦音符
        chord_notes = np.where(chord_sequence[t] > 0)[0]
        
        print(f"\n时间步 {t}:")
        if len(piano_notes) > 0:
            print(f"钢琴音符: {[music21.note.Note(pitch).nameWithOctave for pitch in piano_notes]}")
        else:
            print("钢琴: 休止符")
            
        if len(chord_notes) > 0:
            print(f"和弦音符: {[music21.note.Note(pitch).nameWithOctave for pitch in chord_notes]}")
            # 尝试识别和弦类型
            chord = music21.chord.Chord([music21.note.Note(pitch) for pitch in chord_notes])
            print(f"和弦类型: {chord.commonName}")
        else:
            print("和弦: 无")

def save_to_midi(piano_track, chord_track, output_path='test_sample.mid'):
    """将轨道保存为MIDI文件"""
    # 创建MIDI文件
    midi = music21.midi.MidiFile()
    piano_stream = music21.stream.Stream()
    chord_stream = music21.stream.Stream()
    
    # 添加钢琴音符
    for t in range(len(piano_track)):
        piano_notes = np.where(piano_track[t] > 0)[0]
        if len(piano_notes) > 0:
            for pitch in piano_notes:
                note = music21.note.Note(pitch)
                note.quarterLength = 0.25  # 设置音符长度
                piano_stream.append(note)
    
    # 添加和弦
    for t in range(len(chord_track)):
        chord_notes = np.where(chord_track[t] > 0)[0]
        if len(chord_notes) > 0:
            chord = music21.chord.Chord([music21.note.Note(pitch) for pitch in chord_notes])
            chord.quarterLength = 0.25
            chord_stream.append(chord)
    
    # 合并轨道
    combined_stream = music21.stream.Stream([piano_stream, chord_stream])
    combined_stream.write('midi', fp=output_path)
    print(f"MIDI文件已保存到: {output_path}")

def test_single_file(file_path, start_time=0, length=32):
    """
    测试单个音频文件的和弦旋律分离效果
    
    参数:
    file_path: .npz文件的路径
    start_time: 开始分析的时间点
    length: 要分析的时间步长度
    """
    # 加载数据
    piano_track, chord_track = load_npz_file(file_path)
    
    # 打印轨道信息
    print_track_info(piano_track, chord_track)
    
    # 分析指定时间段
    analyze_sequence(piano_track, chord_track, start_time, length)
    
    # 保存为MIDI文件
    save_to_midi(piano_track, chord_track)
    
    # 尝试播放MIDI文件
    try:
        y, sr = librosa.load('test_sample.mid', sr=None)
        display(Audio(data=y, rate=sr))
    except:
        print("无法播放音频，请使用其他MIDI播放器打开test_sample.mid文件")


In [4]:

"""测试和弦和旋律的分离效果"""
lpd_dir = Path("../data/raw/lpd_5/lpd_5_full/0")

# 获取第一个.npz文件
npz_files = list(lpd_dir.glob("**/*.npz"))

test_file = npz_files[0]
print(f"Testing with file: {test_file}")

data = np.load(test_file)
print(data.files)
# test_single_file(test_file, start_time=0, length=32)

Testing with file: ../data/raw/lpd_5/lpd_5_full/0/04e0d3d701b5e67ccc84887bc6017956.npz
['pianoroll_0_csc_indptr', 'pianoroll_0_csc_indices', 'pianoroll_2_csc_indices', 'pianoroll_2_csc_shape', 'pianoroll_1_csc_indptr', 'pianoroll_3_csc_indptr', 'pianoroll_1_csc_indices', 'pianoroll_3_csc_shape', 'pianoroll_0_csc_data', 'pianoroll_1_csc_data', 'pianoroll_1_csc_shape', 'tempo', 'pianoroll_4_csc_indices', 'pianoroll_3_csc_indices', 'pianoroll_4_csc_shape', 'pianoroll_2_csc_data', 'pianoroll_3_csc_data', 'pianoroll_4_csc_data', 'pianoroll_4_csc_indptr', 'pianoroll_2_csc_indptr', 'downbeat', 'pianoroll_0_csc_shape', 'info.json']


In [16]:
piano_shape = data['pianoroll_1_csc_shape']
piano_indptr = data['pianoroll_1_csc_indptr']
piano_indices = data['pianoroll_1_csc_indices']
piano_data = data['pianoroll_1_csc_data']

# 提取和弦轨道（轨道2）的数据
chord_shape = data['pianoroll_2_csc_shape']
chord_indptr = data['pianoroll_2_csc_indptr']
chord_indices = data['pianoroll_2_csc_indices']
chord_data = data['pianoroll_2_csc_data']

print("\n轨道形状:")
print("钢琴轨道形状:", piano_shape)
print("和弦轨道形状:", chord_shape)


轨道形状:
钢琴轨道形状: [11280   128]
和弦轨道形状: [  0 128]


In [ ]:
import pypianoroll

pianoroll = pypianoroll.load(test_file)

piano_track = pianoroll.tracks[1]
guitar_track = pianoroll.tracks[2]
bass_track = pianoroll.tracks[3]
for t in pianoroll.tracks: 


AttributeError: 'list' object has no attribute 'name'

In [9]:
# make the piano track playable with IPython.display import Audio, display
display(Audio(pianoroll.tracks[1], rate=44100))